In [27]:
import numpy as np
import pandas as pd
import cv2

In [28]:
# bottle_df = pd.DataFrame([])

# for n in range(5):
#     df = pd.read_csv(f"bottle_image_data_{n}.csv")
#     bottle_df = pd.concat([bottle_df, df])

In [29]:
# glass_df = pd.read_csv('glass.csv')
# paper_df = pd.read_csv('paper_all.csv')
# styrofoam_df = pd.read_csv('styrofoam.csv')
# can_df = pd.read_csv('can.csv')
# bottle_df = pd.read_csv('bottle.csv')

In [30]:
# glass_df.drop(columns=glass_df.columns[0], inplace=True)
# bottle_df.drop(columns=glass_df.columns[0], inplace=True)

In [31]:
# bottle_df.drop(columns=bottle_df.columns[0], inplace=True)

In [32]:
# final_ = pd.concat([paper_df, glass_df, styrofoam_df, can_df, bottle_df], ignore_index=True)

In [33]:
# final_.to_csv('final.csv')

In [34]:
final_df = pd.read_csv('final.csv')

In [35]:
final_df.shape

(26939, 22502)

In [36]:
from autogluon.tabular import TabularPredictor

In [37]:
final_part1 = final_df.iloc[:, :5000].astype('int16')
final_part2 = final_df.iloc[:, 5000:10000].astype('int16')
final_part3 = final_df.iloc[:, 10000:15000].astype('int16')
final_part4 = final_df.iloc[:, 15000:20000].astype('int16')
final_part5 = final_df.iloc[:, 20000:-1].astype('int16')
final_part6 = final_df.iloc[:, -1]

In [38]:
final_df.shape

(26939, 22502)

In [39]:
second_ = pd.concat([final_part1, final_part2, final_part3, final_part4, final_part5, final_part6], axis=1, ignore_index=True)

In [40]:
second_.shape

(26939, 22502)

In [41]:
second_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26939 entries, 0 to 26938
Columns: 22502 entries, 0 to 22501
dtypes: int16(22501), object(1)
memory usage: 1.1+ GB


In [42]:
second_.rename(columns={second_.columns[-1]:'label'}, inplace=True)

In [43]:
second_.shape

(26939, 22502)

In [44]:
# second_.to_csv('final_int32.csv')

In [45]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(second_, test_size=0.2,stratify=final_df['label'],random_state=10)

In [46]:
# predictor_RF = TabularPredictor(label='label', path="opencv_model_RF.pkl")

# predictor.fit(train_data=train_data, presets= 'high_quality',  hyperparameters={'GBM': {}, 'XGB': {}, 'RF': {}})
# predictor_LGB.fit(train_data=train_data, presets= 'medium_quality',  hyperparameters={'LGB': {}})
# predictor_RF.fit(train_data=train_data, presets='medium_quality', hyperparameters={'RF': {}})

saved_model_path = r'./opencv_model_RF.pkl'
predictor_RF = TabularPredictor.load(saved_model_path)

In [47]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report
print('================test================')
test_y_ = test_data['label']
pred_y_ = predictor_RF.predict(test_data)

print(accuracy_score(test_y_, pred_y_))
print(classification_report(test_y_,pred_y_))

print('================train================')
train_y_ = train_data['label']
pred_train_y_ = predictor_RF.predict(train_data)

print(accuracy_score(train_y_, pred_train_y_))
print(classification_report(train_y_,pred_train_y_))

================test================
0.9912769116555308
              precision    recall  f1-score   support

      bottle       1.00      0.99      0.99      1000
         can       0.99      0.99      0.99       898
       glass       1.00      1.00      1.00      1200
       paper       0.97      1.00      0.98      1021
   styrofoam       1.00      0.98      0.99      1269

    accuracy                           0.99      5388
   macro avg       0.99      0.99      0.99      5388
weighted avg       0.99      0.99      0.99      5388

================train================
0.9990255672590599
              precision    recall  f1-score   support

      bottle       1.00      1.00      1.00      4000
         can       1.00      1.00      1.00      3590
       glass       1.00      1.00      1.00      4800
       paper       1.00      1.00      1.00      4085
   styrofoam       1.00      1.00      1.00      5076

    accuracy                           1.00     21551
   macro avg      

In [48]:
results = predictor_RF.fit_summary()
print(results) 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0         RandomForest    0.99026       0.255648  79.414343                0.255648          79.414343            1       True          1
1  WeightedEnsemble_L2    0.99026       0.256649  79.418346                0.001001           0.004003            2       True          2
Number of models trained: 2
Types of models trained:
{'WeightedEnsembleModel', 'RFModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('int', []) : 22501 | ['0', '1', '2', '3', '4', ...]
*** End of fit() summary ***
{'model_types': {'RandomForest': 'RFModel', 'WeightedEnsemble_L2': 'WeightedEnsembleModel'}, 'model_performance': {'RandomForest': 0.9902597402597403, 'WeightedEnsemble_L2': 0.9902597402597403}, 'model_best': 'WeightedEnsemble_L2'

c:\Users\KDP-23\anaconda3\envs\mypython\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [52]:
# 이미지 파일 경로
image_path = './서울우유.png'

# 이미지 열기
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# if image is not None:
#     # 이미지 크기 조정
#     resized_img = cv2.resize(image, (150, 150))

#     # 데이터프레임 생성
#     df = pd.DataFrame(resized_img.flatten()).T 
#     df.head()
# else:
#     print("이미지를 열 수 없습니다.")
image

In [ ]:
results = predictor_RF.fit_summary()
